In [7]:
import pandas as pd
import glob
import re
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import csv

# Função utilizada para realizar o merge de arquivos CSV, contidos em um PATH, para um dado PATH
def merge_csv_path(path_in, path_out):
    all_files = glob.glob(path_in + "*.csv")
    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame.to_csv(path_out + 'merge_csv_path.csv')
    
# Indique em path_in a pasta contendo os arquivos CSV a serem juntados, e em path_out a pasta a ser gerada o arquivo concatenado
path_in = 'C:/Users/PC/Desktop/quinto-andar-analise/lista_urls/'
path_out = 'C:/Users/PC/Desktop/quinto-andar-analise/'
# merge_csv_path(path_in, path_out) # Descomente para executar a função.



# Função utilizada para coletar os diversos anúncios do portal QuintoAndar. 
def coleta_de_anuncios(url, loop_de_coleta, path_out):
    option = Options()
    option.headless = True # Se for True, interface gráfica não será aberta
    driver = webdriver.Firefox(executable_path='geckodriver\geckodriver.exe', options=option)

    # Abre a URL
    driver.get(url)

    # Faz o scroll down para mostrar todos os resultados.
    time.sleep(15)
    div_scroll = driver.find_element_by_class_name('sc-5ca2ou-0.TybLn')
    loop = 61
    urls_coletadas = 0

    for i in range(60):
        div_scroll.send_keys(Keys.END)
        time.sleep(1)

    for main in range(loop_de_coleta):
        print("loop: " + str(loop) + " de " + str(rangers))
        loop += 1
        for i in range(10):
            div_scroll.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

        for i in range(7):
            div_scroll.send_keys(Keys.PAGE_UP)
            time.sleep(0.5)

        time.sleep(10)

        # Coleta todas as URLs utilizando a classe "href", que são links
        elems = driver.find_elements_by_xpath("//a[@href]")
        time.sleep(5)
        lista_urls = {'urls': []}

        for elem in elems:
            link_geral = str(elem.get_attribute("href"))
            if ('imovel/8') in link_geral: # O texto 'imovel/8' é um indício de que o link colhido é o de um anúncio de imóvel
                #print(link_geral)
                lista_urls['urls'].append(link_geral)

        # Estruturando conteúdo em diversos arquivos CSV
        df = pd.DataFrame(lista_urls)
        df.to_csv(path_out + str(loop) + '_lista_urls_pandas.csv')
        print('elementos na lista de URLs: ' + str(len(lista_urls['urls'])))
        urls_coletadas = urls_coletadas + len(lista_urls['urls'])
        print(urls_coletadas)

    driver.quit()

# URL contendo os anúncios. Neste caso da cidade de São Paulo
url = 'https://www.quintoandar.com.br/alugar/imovel/sao-paulo-sp-brasil/' 
# Número de loop de coletas. Cada loop colhe cerca de 10 URLs. (Não há como colher todas as URLs geradas, apenas algumas por vez).
loop_de_coleta = 500
# Path que será criado os diversos arquivos CSV contendo as listas de URLs. Utilize a função merge_csv_path para que os arquivos sejam concatenados em um único CSV.
path_out = 'C:/Users/PC/Desktop/quinto-andar-analise/lista_urls/'

#coleta_de_anuncios(url, loop_de_coleta, path_out) # Descomente para chamar a função






